In [56]:
import numpy as np
import pandas as pd 
import subprocess
from pathlib import Path

In [57]:
cmd = '''
cd /home/projects/petero/binding-pocket-project/naccess/test
for filename in /home/projects/petero/binding-pocket-project/naccess/test/*
do
  /home/projects/petero/software/naccess/naccess "$filename";
done
touch SASA_ligand.txt
touch SASA_complex.txt
for filename in test/*
do
  prefix="${filename##*/}"
  base="${prefix%%_*}"
  echo "$base"_ligand_sasa>>SASA_ligand.txt
  grep "CHAIN  1 H" ${base}_ligand.rsa >>SASA_ligand.txt
  echo "$base"_complex_sasa >>SASA_complex.txt
  grep "CHAIN  2 H" ${base}_complex.rsa >>SASA_complex.txt
done 
'''

In [58]:
subprocess.run(cmd, shell=True, check=True, executable='/usr/bin/bash')


CompletedProcess(args='\ncd /home/projects/petero/binding-pocket-project/naccess/test\nfor filename in /home/projects/petero/binding-pocket-project/naccess/test/*\ndo\n  /home/projects/petero/software/naccess/naccess "$filename";\ndone\ntouch SASA_ligand.txt\ntouch SASA_complex.txt\nfor filename in test/*\ndo\n  prefix="${filename##*/}"\n  base="${prefix%%_*}"\n  echo "$base"_ligand_sasa>>SASA_ligand.txt\n  grep "CHAIN  1 H" ${base}_ligand.rsa >>SASA_ligand.txt\n  echo "$base"_complex_sasa >>SASA_complex.txt\n  grep "CHAIN  2 H" ${base}_complex.rsa >>SASA_complex.txt\ndone \n', returncode=0)

In [61]:
df = pd.read_fwf('/home/projects/petero/binding-pocket-project/naccess/test/SASA_ligand.txt', skiprows=1, header = None)
df1 = pd.read_fwf('/home/projects/petero/binding-pocket-project/naccess/test/SASA_complex.txt', skiprows=1, header = None)

In [62]:
df

,0,1,2,3,4,5,6,7
0,17000_ligand.rsa:CHAIN,1,H,671.9,671.9,0.0,613.8,58.1
1,20000_ligand.rsa:CHAIN,1,H,665.1,665.1,0.0,606.3,58.8


In [63]:
df1

,0,1,2,3,4,5,6,7
0,17000_complex.rsa:CHAIN,2,H,198.9,198.9,0.0,185.8,13.1
1,20000_complex.rsa:CHAIN,2,H,201.5,201.5,0.0,181.9,19.7


In [64]:
df[0] = df[0].str.extract('([^_]+)')
df1[0] = df1[0].str.extract('([^_]+)')

In [65]:
df = df.drop(df.columns[[1, 2]], axis=1)
df1 = df1.drop(df1.columns[[1, 2]], axis=1)

In [66]:
df.columns= ['PDB ID', 'All-atoms', 'Total-Side', 'Main-Chain', 'Non-polar', 'All polar']
df1.columns= ['PDB ID','All-atoms', 'Total-Side', 'Main-Chain', 'Non-polar', 'All polar']

In [67]:
df

,PDB ID,All-atoms,Total-Side,Main-Chain,Non-polar,All polar
0,17000,671.9,671.9,0.0,613.8,58.1
1,20000,665.1,665.1,0.0,606.3,58.8


In [68]:
df1

,PDB ID,All-atoms,Total-Side,Main-Chain,Non-polar,All polar
0,17000,198.9,198.9,0.0,185.8,13.1
1,20000,201.5,201.5,0.0,181.9,19.7


In [69]:
df2 = (pd.merge(df, df1, on='PDB ID'))

In [70]:
df2

,PDB ID,All-atoms_x,Total-Side_x,Main-Chain_x,Non-polar_x,All polar_x,All-atoms_y,Total-Side_y,Main-Chain_y,Non-polar_y,All polar_y
0,17000,671.9,671.9,0.0,613.8,58.1,198.9,198.9,0.0,185.8,13.1
1,20000,665.1,665.1,0.0,606.3,58.8,201.5,201.5,0.0,181.9,19.7


In [71]:
df2['BSA_total'] = df2['All-atoms_x']- df2['All-atoms_y']
df2['MESA_total'] = df2['All-atoms_y']

In [72]:
df2['BSA_nonpolar'] = df2['Non-polar_x']- df2['Non-polar_y']
df2['MESA_nonpolar'] = df2['Non-polar_y']

In [73]:
df2['BSA_polar'] = df2['All polar_x']- df2['All polar_y']
df2['MESA_polar'] = df2['All polar_y']

In [74]:
df2

,PDB ID,All-atoms_x,Total-Side_x,Main-Chain_x,Non-polar_x,All polar_x,All-atoms_y,Total-Side_y,Main-Chain_y,Non-polar_y,All polar_y,BSA_total,MESA_total,BSA_nonpolar,MESA_nonpolar,BSA_polar,MESA_polar
0,17000,671.9,671.9,0.0,613.8,58.1,198.9,198.9,0.0,185.8,13.1,473.0,198.9,428.0,185.8,45.0,13.1
1,20000,665.1,665.1,0.0,606.3,58.8,201.5,201.5,0.0,181.9,19.7,463.6,201.5,424.4,181.9,39.1,19.7


In [75]:
df2.to_csv('/home/projects/petero/binding-pocket-project/naccess/test/final_sasa.csv', index=False)